In [123]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [124]:
datasetlabel = pd.read_csv('../../data/datasetlabel.csv') # non stationary data
fed_funds = pd.read_csv('../../data/1m-fed_fund_futures.csv') # fed funds data
other_macro = pd.read_csv('../../data/mdpurg9fqkqhobkm.csv') # CPI and yields
fracdiff_data = pd.read_csv('../../data/DATA_FINAL.csv') # fracdiff data
alternative_dataset = pd.read_csv('../../data/fixed_divyield.csv') # Ettore's data

The base is fracdiff_data, which contains the fractionally differentiated data. From that we add the features from fed_funds, other_macro and alternative_dataset. We then .diff() all the features which are not differentiated. 

In [125]:
# Interpolating the data from fracdiff_data 

fracdiff_data = fracdiff_data[fracdiff_data['date'] >= '2008-01-01']
fracdiff_data = fracdiff_data.interpolate(method='linear')

C:\Users\Asus\AppData\Local\Temp\ipykernel_1504\926487017.py:4: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  fracdiff_data = fracdiff_data.interpolate(method='linear')


In [126]:
# test to see if the interpolation worked
np.sum(fracdiff_data.isna().sum(axis=1) > 0)

0

We first merge all the non differentiated features into one and then run .diff() on them 

In [127]:
# convert every 'Date' to datetime
datasetlabel['date'] = pd.to_datetime(datasetlabel['date'])
fed_funds['Date'] = pd.to_datetime(fed_funds['Date'])
fed_funds.rename(columns={'Date': 'date'}, inplace=True)
other_macro['caldt'] = pd.to_datetime(other_macro['caldt'])
other_macro.rename(columns={'caldt': 'date'}, inplace=True)
fracdiff_data['date'] = pd.to_datetime(fracdiff_data['date'])
alternative_dataset['date'] = pd.to_datetime(alternative_dataset['date'])
alternative_dataset['date'] = pd.to_datetime(alternative_dataset['date'])

In [128]:
fed_funds = fed_funds[['date', 'Adj Close', 'Volume']]
fed_funds.rename(columns={'Adj Close': 'fed_funds_adj_close', 'Volume': 'fed_funds_volume'}, inplace=True)

# merge fed_funds with other_macro

# CAN'T USE OTHER_MACRO BECAUSE IT STARTS IN 2009
#macro_data = pd.merge(fed_funds, other_macro, on='date', how='inner')

In [129]:
set(datasetlabel.columns) - set(fracdiff_data.columns)

{'stat_divyeld', 'target'}

In [130]:
alternative_dataset = pd.read_csv('../../data/fixed_divyield.csv') # Ettore's data

In [131]:
# keep only the columns of alternative_dataset that are not in fracdiff_data
cols_to_keep = ['date', 'permno', '12_month_return', '3_month_return']
alternative_dataset = alternative_dataset[cols_to_keep]
alternative_dataset.columns

Index(['date', 'permno', '12_month_return', '3_month_return'], dtype='object')

In [132]:
print(np.sum(alternative_dataset[alternative_dataset['date']>='2008-01-01'].isna().sum(axis=1) > 0))

# find out the columsn that have the most missing values in alternative_dataset[alternative_dataset['date']>='2008-01-01']
alternative_dataset[alternative_dataset['date']>='2008-01-01'].isna().sum().sort_values(ascending=False)

27090


12_month_return    27090
3_month_return      6679
date                   0
permno                 0
dtype: int64

In [136]:
datasetlabel = datasetlabel[['date', 'permno', 'stat_divyeld']]
datasetlabel['date'] = pd.to_datetime(datasetlabel['date'])
alternative_dataset['date'] = pd.to_datetime(alternative_dataset['date'])

In [137]:
# join the single feature of datasetlabel with alternative_dataset on 'date' and 'permno'
non_diff_dataset = pd.merge(datasetlabel, alternative_dataset, on=['date', 'permno'], how='inner')

In [138]:
# merge non_diff_dataset with fed_funds on date
non_diff_dataset = pd.merge(non_diff_dataset, fed_funds, on='date', how='inner')

# interpolate the missing values in non_diff_dataset
non_diff_dataset = non_diff_dataset.interpolate(method='linear')

In [139]:
# run .diff() on the columns that are not 'date' and 'permno' of non_diff_dataset
diff_dataset = non_diff_dataset.copy()
diff_dataset[diff_dataset.columns.difference(['date', 'permno'])] = non_diff_dataset[non_diff_dataset.columns.difference(['date', 'permno'])].diff()

In [140]:
diff_dataset.dropna(inplace=True) # drop the first row of NaNs since it's not possible to calculate the difference

In [147]:
# merge diff_dataset with fracdiff_data on 'date' and 'permno'
definitive_dataset = pd.merge(diff_dataset, fracdiff_data, on=['date', 'permno'], how='inner')

In [ ]:
# adding the target variable to the definitive_dataset

In [149]:
definitive_dataset.to_csv('../../data/definitive_dataset.csv', index=False)